In [28]:
import json
import os
import openpyxl
import jieba
import pandas as pd
import time
from prettytable import PrettyTable
# import inspect, re
import numpy as np
'''获取词频，获取字频，给词频和字频附带分类个数'''
words = {}
characters = {}
valid_papers = {}
invalid_papers = {}
# 得分
valid_words_score_list = []
valid_characters_score_list = []
invalid_words_score_list = []
invalid_characters_score_list = []
# 函数计时器
timer_split_and_insert_count = 0
timer_scoring_count = 0
timer_item_process_count = 0

# def varname(p):
# # 获取变量名
#     for line in inspect.getframeinfo(inspect.currentframe().f_back)[3]:
#         m = re.search(r'\bvarname\s*\(\s*([A-Za-z_][A-Za-z0-9_]*)\s*\)', line)
#     if m:
#         return m.group(1)

def timer_split_and_insert(func):
    def func_in(*s,**gs):
        start_time = time.time()
        _res_ = func(*s,**gs)
        end_time = time.time()
        global timer_split_and_insert_count
        timer_split_and_insert_count += end_time - start_time
        return  _res_
    return  func_in

def timer_scoring(func):
    def func_in(*s,**gs):
        start_time = time.time()
        _res_ = func(*s,**gs)
        end_time = time.time()
        global timer_scoring_count
        timer_scoring_count += end_time - start_time
        return  _res_
    return  func_in

def timer_item_process(func):
    def func_in(*s,**gs):
        start_time = time.time()
        _res_ = func(*s,**gs)
        end_time = time.time()
        global timer_item_process_count
        timer_item_process_count += end_time - start_time
        return  _res_
    return  func_in

def corpus(corpus_path, exception_path):
    def exception_list(exception_path):
        _exception_list_ = []
        with open(exception_path, 'r', encoding='utf-8') as f:
            data = f.readlines()
            for index in range(len(data)):
                data[index] = str(data[index][:-1])
            _exception_list_.extend(data)
        return _exception_list_

    filenames = os.listdir(corpus_path)
    _corpus_ = []
    for filename in filenames:
        try:
            with open(corpus_path+r"/"+filename, 'r', encoding='gbk') as f:
                data = f.readlines()
                for index in range(len(data)):
                    data[index] = data[index].strip()
                _corpus_.extend(data)
        except:
            with open(corpus_path+r"/"+filename, 'r', encoding='utf-8') as f:
                data = f.readlines()
                for index in range(len(data)):
                    data[index] = data[index].strip()
                _corpus_.extend(data)
    # 删除 exception list 中的单词
    _corpus_exception_list_ = exception_list(exception_path)
    for _corpus_exception_word_ in _corpus_exception_list_:
        try:
            _corpus_.__delitem__(_corpus_.index(_corpus_exception_word_))
        except:
            pass
    return _corpus_

@timer_split_and_insert
def split_and_insert(item, ZhongYi_or_not):
    def valid(strs):
        #检验是否全是中文字符
        for _char in strs:
            if not '\u4e00' <= _char <= '\u9fa5':
                return False
        return True

    def word_insert(word, ZhongYi_or_not):
        # 对words字典中的对应key进行value的增加
        if word in words:
            words[word] = [words[word][0]+1, words[word][1]+ZhongYi_or_not]
        else:
            words[word] = [1, 0+ZhongYi_or_not]

    def character_insert(character, ZhongYi_or_not):
        if character in characters:
            characters[character] = [characters[character][0]+1, characters[character][1]+ZhongYi_or_not]
        else:
            characters[character] = [1, 0+ZhongYi_or_not]

    json_keys = ["title", "abstract", "keywords"]
    for json_key in json_keys:
        item_words = jieba.lcut(item[json_key])
        for item_word in item_words:
            if valid(item_word):
                word_insert(item_word, ZhongYi_or_not)
        for cha in item[json_key]:
            if valid(cha):
                character_insert(cha, ZhongYi_or_not)

@timer_scoring
def scoring(item, excel_words, excel_characters, common_chinese_characters, score_method = 'words'):
    item_text = item["title"] + "。" + item["abstract"] + "。" + item["keywords"]
    score = 0
    common_chinese_characters_weight = 1/5
    common_chinese_words_weight = 1/3
    if score_method == 'words':
        # 按单词打分
        item_text_WordsOrCharacters = jieba.lcut(item_text)
        prob_dict = excel_words
    else:
        # 按字符打分
        item_text_WordsOrCharacters = item_text
        prob_dict = excel_characters

    for item_text_WordOrCharacter in item_text_WordsOrCharacters:
        # 舍弃特殊字符（标点符号等）。若单词或字符为中文//英文//数字，isalnum()函数则为True
        if item_text_WordOrCharacter.isalnum():
            # 判断是否为中文
            is_Chinese = True
            for _char in item_text_WordOrCharacter:
                if not '\u4e00' <= _char <= '\u9fa5':
                    is_Chinese = False
                    break
            # 若单词或字符为英文或数字，则减1分
            if not is_Chinese:
                score -= 1
                continue
            # 判断是否包含不常用汉字
            is_common_chinese = True
            for _char in item_text_WordOrCharacter:
                if _char not in common_chinese_characters:
                    is_common_chinese = False
                    break
            try:
                if is_common_chinese:
                    # 若为常用中文单词，则将单词概率的1/3或1/5作为分数。
                    score += prob_dict[item_text_WordOrCharacter]['概率']*\
                        (common_chinese_characters_weight*(1-(score_method=='words'))+\
                         common_chinese_words_weight*(score_method=='words'))
                else:
                    # 若单词包含不常用汉字字符，则将概率作为分数
                    score += prob_dict[item_text_WordOrCharacter]['概率']
            except:
                # 舍弃excel中不包含的单词
                pass
    score /= len(item_text_WordsOrCharacters)
    return score

def wordFrequency(json_path, zhongyi_corpus, excel_words, excel_characters, common_chinese_characters):
    # 读取文章json
    with open(json_path, 'r',encoding='utf-8') as f:
        items_json = json.load(f)
    # 优先处理的条目
    most_common_word_list = ["中医", "中药", "中医药", "中西医"]
    # 中医文章的数量
    valid_num = 0

    def check_and_checkGrammar(word, item_text):
        if word in item_text:
            if word in jieba.lcut(item_text):
                return True
        return False

    @timer_item_process
    def item_process(item, identify_word, valid_or_not):
        split_and_insert(item, valid_or_not)
        words_score = scoring(item, excel_words, excel_characters, common_chinese_characters, score_method='words')
        characters_score = scoring(item, excel_words, excel_characters, common_chinese_characters, score_method='characters')
        if valid_or_not:
            valid_papers[item["title"]] = {
                "abstract": item["abstract"],
                "keywords": item["keywords"],
                "identify_word": identify_word,
                "words_score": words_score,
                "characters_score": characters_score,
                "valid": valid_or_not,
            }
            valid_words_score_list.append(words_score)
            valid_characters_score_list.append(characters_score)
        else:
            invalid_papers[item["title"]] = {
                "abstract": item["abstract"],
                "keywords": item["keywords"],
                "identify_word": identify_word,
                "words_score": words_score,
                "characters_score": characters_score,
                "valid": valid_or_not,
            }
            invalid_words_score_list.append(words_score)
            invalid_characters_score_list.append(characters_score)

    for item in items_json:
        # 跳过非条目项
        if "title" not in item:
            continue
        item_text = item["title"] + "。" + item["abstract"] + "。" + item["keywords"]
        continue_tag = False
        # 处理最常见的单词
        for most_common_word in most_common_word_list:
            if check_and_checkGrammar(most_common_word, item_text):
                valid_num += 1
                item_process(item, most_common_word, True)
                continue_tag = True
                break
        if continue_tag:
            continue
        # 遍历整个corpus，寻找item中出现的单词
        for word in zhongyi_corpus:
            if word in item_text:
                valid_num += 1
                item_process(item, word, True)
                continue_tag = True
                break
        if continue_tag:
            continue
        # 本篇文章未找到中医关键词
        item_process(item, "", False)
    print(json_path, "：", (len(items_json), valid_num))
    return len(items_json), valid_num

def save_Frequency(database_path):
    def save_to(entry, filename):
        try:
            # utf-8将一个汉字编码为3个字节,gbk将一个汉字编码为2个字节,
            with open(filename, 'w',encoding='utf-8') as new_f:  # 重新写入
                json.dump(entry, new_f,ensure_ascii=False,indent=4) # 最后一个参数会保证dump之后的结果所有的字符都能被ascii表示
        except json.decoder.JSONDecodeError:
            print('Make sure the json file is in valid format,[{},{}...]')
    # 保存到json
    save_to(words, database_path + '/' + r"words.json")
    save_to(characters, database_path +'/' + r"characters.json")
    # 保存到excel
    workbook = openpyxl.Workbook()
    word_sheet = workbook.active
    word_sheet.title = 'words'
    for word in words.keys():
        word_sheet.append([word, words[word][0], words[word][1]])
    character_sheet = workbook.create_sheet(index=1, title="characters")
    for character in characters.keys():
        character_sheet.append([character, characters[character][0], characters[character][1]])
    workbook.save(database_path + "/" + 'frequency.xlsx')

def save_Json(database_path):
    def save_to(entry, filename):
        try:
            # utf-8将一个汉字编码为3个字节,gbk将一个汉字编码为2个字节,
            with open(filename, 'w',encoding='utf-8') as new_f:  # 重新写入
                json.dump(entry, new_f,ensure_ascii=False,indent=4) # 最后一个参数会保证dump之后的结果所有的字符都能被ascii表示
        except json.decoder.JSONDecodeError:
            print('Make sure the json file is in valid format,[{},{}...]')
    save_to(valid_papers, database_path + '/' + 'valid_papers.json')
    save_to(invalid_papers, database_path + '/' + 'invalid_papers.json')

def main():
    json_path = r"D://vscode_workspace//ZhongYiPapers//database//分省份数据"
    # json_path = r"D://vscode_workspace//ZhongYiPapers//database//TMP"
    save_path = r"D://vscode_workspace//ZhongYiPapers//database//12_24新处理"
    common_chinese_characters_path = r"D://vscode_workspace//ZhongYiPapers//词汇库//common_chinese_characters.json"
    prob_excel_path = r"D://vscode_workspace//ZhongYiPapers//database//12_23新处理//frequency.xlsx"
    exception_path = r"D://vscode_workspace//ZhongYiPapers//词汇库//不包含的词汇.txt"
    corpus_path = r"D://vscode_workspace//ZhongYiPapers//词汇库//处理完成"
    # 读取常用汉字
    with open(common_chinese_characters_path, 'r',encoding='utf-8') as f:
        common_chinese_characters = json.load(f)
        common_chinese_characters = set(common_chinese_characters)
    # 读取词频
    excel_words = pd.DataFrame(pd.read_excel(prob_excel_path, sheet_name='words', header=0, index_col=0))
    excel_words = excel_words.to_dict('index')
    excel_characters = pd.DataFrame(pd.read_excel(prob_excel_path, sheet_name='characters', header=0, index_col=0))
    excel_characters = excel_characters.to_dict('index')
    # 读取中医词库
    zhongyi_corpus = corpus(corpus_path, exception_path)
    # 读取需要处理的json列表
    json_names = os.listdir(json_path)
    # 依次处理json
    total_json_num = 0
    total_valid_num = 0
    for json_name in json_names:
        json_len, valid_num = wordFrequency(json_path+'/'+json_name, zhongyi_corpus, excel_words, excel_characters, common_chinese_characters)
        total_json_num += json_len
        total_valid_num += valid_num
    save_Frequency(save_path)
    save_Json(save_path)

    table = PrettyTable(['Total item num','Total valid num','Ratio'])
    table.add_row([total_json_num, total_valid_num, total_valid_num/total_json_num])
    print(table)

    words_score_list = valid_words_score_list + invalid_words_score_list
    characters_score_list = valid_characters_score_list + invalid_characters_score_list
    table = PrettyTable(['Variant','Mean','Var'])
    table.add_row(["Words", np.mean(words_score_list), np.var(words_score_list)])
    table.add_row(["Valid words", np.mean(valid_words_score_list), np.var(valid_words_score_list)])
    table.add_row(["Invalid words", np.mean(invalid_words_score_list), np.var(invalid_words_score_list)])
    table.add_row(["Characters", np.mean(characters_score_list), np.var(characters_score_list)])
    table.add_row(["Valid characters", np.mean(valid_characters_score_list), np.var(valid_characters_score_list)])
    table.add_row(["Invalid characters", np.mean(invalid_characters_score_list), np.var(invalid_characters_score_list)])
    print(table)

    table = PrettyTable(['Function','Time cost'])
    table.add_row(["split_and_insert", timer_split_and_insert_count])
    table.add_row(["scoring", timer_scoring_count])
    table.add_row(["item_process", timer_item_process_count])
    print(table)

main()

D://vscode_workspace//ZhongYiPapers//database//分省份数据/papers_lzd.json ： (44146, 31755)
D://vscode_workspace//ZhongYiPapers//database//分省份数据/result2010.json ： (25094, 17471)
D://vscode_workspace//ZhongYiPapers//database//分省份数据/result2011.json ： (27254, 19122)
D://vscode_workspace//ZhongYiPapers//database//分省份数据/result2012.json ： (29984, 20804)
D://vscode_workspace//ZhongYiPapers//database//分省份数据/result2013.json ： (32436, 22725)
D://vscode_workspace//ZhongYiPapers//database//分省份数据/result2014.json ： (33639, 23713)
D://vscode_workspace//ZhongYiPapers//database//分省份数据/result2015.json ： (32814, 22993)
D://vscode_workspace//ZhongYiPapers//database//分省份数据/result2016.json ： (32418, 23068)
D://vscode_workspace//ZhongYiPapers//database//分省份数据/result2017.json ： (32120, 23007)
D://vscode_workspace//ZhongYiPapers//database//分省份数据/result2018.json ： (31620, 22458)
D://vscode_workspace//ZhongYiPapers//database//分省份数据/result2019.json ： (32706, 23283)
D://vscode_workspace//ZhongYiPapers//database//分省份数据/r